# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 16 2022 11:42AM,239756,19,DEX0007003,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
1,Jun 16 2022 11:42AM,239756,19,DEX0007004,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
2,Jun 16 2022 11:42AM,239756,19,DEX0007005,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
3,Jun 16 2022 11:38AM,239755,16,SAR00000342_CD,Sartorius Stedim North America,Released
4,Jun 16 2022 11:36AM,239754,16,SAR00000342,Sartorius Stedim North America,Released
5,Jun 16 2022 11:35AM,239753,22,618086,"NBTY Global, Inc.",Released
6,Jun 16 2022 11:34AM,239752,19,ACG-2102490351,ACG North America LLC,Released
7,Jun 16 2022 11:33AM,239751,10,8148029,Eywa Pharma Inc.,Released
8,Jun 16 2022 11:33AM,239751,10,8148056,Eywa Pharma Inc.,Released
9,Jun 16 2022 11:33AM,239751,10,8148057,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
39,239752,Released,1
40,239753,Released,1
41,239754,Released,1
42,239755,Released,1
43,239756,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239752,NaN,NaN,1.0
239753,NaN,NaN,1.0
239754,NaN,NaN,1.0
239755,NaN,NaN,1.0
239756,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239654,21.0,52.0,1.0
239659,0.0,0.0,1.0
239660,0.0,0.0,1.0
239665,0.0,1.0,0.0
239672,0.0,0.0,29.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239654,21,52,1
239659,0,0,1
239660,0,0,1
239665,0,1,0
239672,0,0,29


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239654,21,52,1
1,239659,0,0,1
2,239660,0,0,1
3,239665,0,1,0
4,239672,0,0,29


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239654,21,52,1
1,239659,,,1
2,239660,,,1
3,239665,,1,
4,239672,,,29


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 16 2022 11:42AM,239756,19,Dexcel Pharma Technologies Ltd. DBA WDSrx
3,Jun 16 2022 11:38AM,239755,16,Sartorius Stedim North America
4,Jun 16 2022 11:36AM,239754,16,Sartorius Stedim North America
5,Jun 16 2022 11:35AM,239753,22,"NBTY Global, Inc."
6,Jun 16 2022 11:34AM,239752,19,ACG North America LLC
7,Jun 16 2022 11:33AM,239751,10,Eywa Pharma Inc.
14,Jun 16 2022 11:22AM,239747,20,"ACI Healthcare USA, Inc."
27,Jun 16 2022 11:08AM,239750,12,LF of America Corp.
28,Jun 16 2022 10:54AM,239749,12,Proven Skincare
29,Jun 16 2022 10:50AM,239748,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 16 2022 11:42AM,239756,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,,,3
1,Jun 16 2022 11:38AM,239755,16,Sartorius Stedim North America,,,1
2,Jun 16 2022 11:36AM,239754,16,Sartorius Stedim North America,,,1
3,Jun 16 2022 11:35AM,239753,22,"NBTY Global, Inc.",,,1
4,Jun 16 2022 11:34AM,239752,19,ACG North America LLC,,,1
5,Jun 16 2022 11:33AM,239751,10,Eywa Pharma Inc.,,,7
6,Jun 16 2022 11:22AM,239747,20,"ACI Healthcare USA, Inc.",,,13
7,Jun 16 2022 11:08AM,239750,12,LF of America Corp.,,,1
8,Jun 16 2022 10:54AM,239749,12,Proven Skincare,,,1
9,Jun 16 2022 10:50AM,239748,19,"AdvaGen Pharma, Ltd",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 16 2022 11:42AM,239756,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,3,,
1,Jun 16 2022 11:38AM,239755,16,Sartorius Stedim North America,1,,
2,Jun 16 2022 11:36AM,239754,16,Sartorius Stedim North America,1,,
3,Jun 16 2022 11:35AM,239753,22,"NBTY Global, Inc.",1,,
4,Jun 16 2022 11:34AM,239752,19,ACG North America LLC,1,,
5,Jun 16 2022 11:33AM,239751,10,Eywa Pharma Inc.,7,,
6,Jun 16 2022 11:22AM,239747,20,"ACI Healthcare USA, Inc.",13,,
7,Jun 16 2022 11:08AM,239750,12,LF of America Corp.,1,,
8,Jun 16 2022 10:54AM,239749,12,Proven Skincare,1,,
9,Jun 16 2022 10:50AM,239748,19,"AdvaGen Pharma, Ltd",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 16 2022 11:42AM,239756,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,3,,
1,Jun 16 2022 11:38AM,239755,16,Sartorius Stedim North America,1,,
2,Jun 16 2022 11:36AM,239754,16,Sartorius Stedim North America,1,,
3,Jun 16 2022 11:35AM,239753,22,"NBTY Global, Inc.",1,,
4,Jun 16 2022 11:34AM,239752,19,ACG North America LLC,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 16 2022 11:42AM,239756,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,3.0,NaN,NaN
1,Jun 16 2022 11:38AM,239755,16,Sartorius Stedim North America,1.0,NaN,NaN
2,Jun 16 2022 11:36AM,239754,16,Sartorius Stedim North America,1.0,NaN,NaN
3,Jun 16 2022 11:35AM,239753,22,"NBTY Global, Inc.",1.0,NaN,NaN
4,Jun 16 2022 11:34AM,239752,19,ACG North America LLC,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 16 2022 11:42AM,239756,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,3.0,0.0,0.0
1,Jun 16 2022 11:38AM,239755,16,Sartorius Stedim North America,1.0,0.0,0.0
2,Jun 16 2022 11:36AM,239754,16,Sartorius Stedim North America,1.0,0.0,0.0
3,Jun 16 2022 11:35AM,239753,22,"NBTY Global, Inc.",1.0,0.0,0.0
4,Jun 16 2022 11:34AM,239752,19,ACG North America LLC,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1917729,83.0,1.0,7.0
12,719245,3.0,1.0,0.0
16,719208,3.0,0.0,0.0
18,239686,1.0,0.0,0.0
19,1438329,15.0,3.0,30.0
20,2157270,58.0,63.0,21.0
22,958768,4.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1917729,83.0,1.0,7.0
1,12,719245,3.0,1.0,0.0
2,16,719208,3.0,0.0,0.0
3,18,239686,1.0,0.0,0.0
4,19,1438329,15.0,3.0,30.0
5,20,2157270,58.0,63.0,21.0
6,22,958768,4.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,83.0,1.0,7.0
1,12,3.0,1.0,0.0
2,16,3.0,0.0,0.0
3,18,1.0,0.0,0.0
4,19,15.0,3.0,30.0
5,20,58.0,63.0,21.0
6,22,4.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,83.0
1,12,Released,3.0
2,16,Released,3.0
3,18,Released,1.0
4,19,Released,15.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,18,19,20,22
Status,,,,,,,
Completed,7.0,0.0,0.0,0.0,30.0,21.0,0.0
Executing,1.0,1.0,0.0,0.0,3.0,63.0,0.0
Released,83.0,3.0,3.0,1.0,15.0,58.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,18,19,20,22
0,Completed,7.0,0.0,0.0,0.0,30.0,21.0,0.0
1,Executing,1.0,1.0,0.0,0.0,3.0,63.0,0.0
2,Released,83.0,3.0,3.0,1.0,15.0,58.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,18,19,20,22
0,Completed,7.0,0.0,0.0,0.0,30.0,21.0,0.0
1,Executing,1.0,1.0,0.0,0.0,3.0,63.0,0.0
2,Released,83.0,3.0,3.0,1.0,15.0,58.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()